<a href="https://colab.research.google.com/github/digodox/Imersao_Alura_Google_AI/blob/main/Projeto_Imersao_Alura_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importação das bibliotecas

In [76]:
import hashlib
from IPython.display import display
from IPython.display import Markdown
import textwrap
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

##Definição dos caminhos das planilhas de dados

In [77]:
path_dados_racionalização = "/content/sample_data/Dados/Racionalizacao_de_alarmes.xlsx"
path_dados_alarmes = "/content/sample_data/Dados/Alarmes.xlsx"
path_dados_tags_analogicos = "/content/sample_data/Dados/TAGs_Analogicos.xlsx"
path_dados_limites_operacionais = "/content/sample_data/Dados/Limites Operacionais.xlsx"

##Configurando a API Key

In [78]:
api_key = userdata.get("GENAI_API_KEY")
genai.configure(api_key=api_key)

##Configuração do modelo

In [79]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Aja como um engenheiro."

##Markdown

In [80]:
def to_markdown(text):
  text = text.replace("'", " *")
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##Extração dos dados das planilhas e preparando o histórico do chat

In [81]:
def extrair_dados_racionalizacao(alarme):
  tagname = alarme['Tagname']
  descricao = alarme['Descrição do Alarme']
  prioridade = alarme['Prioridade']
  causa = alarme['Causa iniciadora']
  acao = alarme['Ação da Central de Controle']
  impacto = alarme['Impacto direto']
  consequencia = alarme['Consequência Potencial']
  supressao = alarme['Supressão Manual']

  dado_extraido = f"O alarme '{tagname}', que se refere à '{descricao}', e considerando que a causa iniciadora é a '{causa}', e com prioridade '{prioridade}'."
  return dado_extraido

def extrair_dados_alarmes(alarme):
  tagname = alarme['Tagname']
  status = alarme['Status do Alarme']

  dado_extraido = f"O alarme '{tagname}' está no estado '{status}'."
  return dado_extraido

def extrair_dados_valores_anal(alarme):
  tagname = alarme['TagName']
  description = alarme['Description']
  value = alarme['Value']
  units = alarme['Units']

  dado_extraido = f"O tag '{tagname}', que se refere à '{description}', possui o valor atual de '{value}' '{units}'."
  return dado_extraido

def extrair_dados_limites_operacionais(alarme):
  estacao = alarme['Estação']
  variavel = alarme['Variável']
  limite_superior = alarme['Limite Superior']
  limite_inferior = alarme['Limite Inferior']
  units = alarme['Units']

  dado_extraido = f"A variável '{variavel}', que se refere à estação '{estacao}', possui limite máximo de '{limite_superior}' '{units}' e limite mínimo de '{limite_inferior}' '{units}'."
  return dado_extraido

##Definição do contexto inicial na variável history[]

In [82]:
#Chat history variable
history = []

df = pd.read_excel(path_dados_racionalização)
for index, row in df.iterrows():
  alarme = row.to_dict()
  dados_racionalizacao = extrair_dados_racionalizacao(alarme)
  history.append({
      "role": "user",
      "parts": dados_racionalizacao
      })

df = pd.read_excel(path_dados_alarmes)
for index, row in df.iterrows():
  alarme = row.to_dict()
  dados_alarmes = extrair_dados_alarmes(alarme)
  history.append({
      "role": "user",
      "parts": dados_alarmes
      })

df = pd.read_excel(path_dados_tags_analogicos)
for index, row in df.iterrows():
  alarme = row.to_dict()
  dados_tags_analogicos = extrair_dados_valores_anal(alarme)
  history.append({
    "role": "user",
    "parts": dados_tags_analogicos
    })

df = pd.read_excel(path_dados_limites_operacionais)
for index, row in df.iterrows():
  alarme = row.to_dict()
  dados_limites_operacionais = extrair_dados_limites_operacionais(alarme)
  history.append({
    "role": "user",
    "parts": dados_limites_operacionais
    })

##Inicialização do modelo

In [83]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

## Chat prompt

Sugestão de perguntas para o modelo:

*   Sumarize o status das estações.
*   Qual o status da estação 01.
*   Quais ações tomar sobre o alarme crítico da estação 01.
*   Digite "fim" no prompt para encerrar o chat.

In [84]:
convo = model.start_chat(history=history)

prompt = input("Digite sua pergunta: ")
while prompt != "fim":
  convo.send_message(prompt)
  print("Resposta: ")
  display(to_markdown(convo.last.text))
  print("\n")
  prompt = input("Digite sua pergunta: ")


Digite sua pergunta: Sumarize o status das estações
Resposta: 


> ## Sumário do Estado das Estações CITY
> 
> **Alarmes:**
> 
> * **CITY_05:** Nível de água no aquecedor B - atuado (prioridade baixa)
> * **CITY_01:** Válvula de bloqueio - Entrada do aquecedor A - Falha - atuado (prioridade crítica)
> * **CITY_02:** Válvula VES-410 Aberta - atuado (prioridade crítica)
> * **CITY_07:** Válvula XV-014B Fechada - atuado (prioridade alta)
> 
> **Variáveis Fora dos Limites:**
> 
> * **CITY_02:** Vazão volumétrica total (411.56 km³/d) - acima do limite máximo de 100 km³/d.
> * **CITY_02:** Pressão de saída (49.83 kgf/cm²) - acima do limite máximo de 35 kgf/cm².
> 
> **Observações:**
> 
> * Todas as outras estações (CITY_01, CITY_03, CITY_04, CITY_06) estão com seus alarmes em estado normal e variáveis dentro dos limites operacionais. 
> * A estação **CITY_05** apresenta um alarme de nível baixo de água no aquecedor B, porém com prioridade baixa. Investigar a causa (falta de água na caixa d *água da estação ou problema no instrumento).
> * As estações **CITY_01**, **CITY_02**, e **CITY_07** apresentam alarmes de alta prioridade. **Acionar equipe de manutenção imediatamente** para investigar as causas:
>     * **CITY_01:** Falha na válvula de bloqueio de entrada do aquecedor A.
>     * **CITY_02:** Válvula VES-410 aberta, vazão e pressão de saída acima dos limites. 
>     * **CITY_07:** Válvula XV-014B fechada.
> 
> **Recomendações:**
> 
> * Priorizar a resolução dos alarmes críticos e alta prioridade nas estações **CITY_01, CITY_02, e CITY_07**.
> * Monitorar a situação do nível de água no aquecedor B na estação **CITY_05**.
> * Verificar a calibração dos instrumentos de medição de vazão e pressão na estação **CITY_02**.
> 
> Este sumário visa auxiliar na rápida tomada de decisão e priorização de ações para garantir a operação segura e eficiente das estações. 




Digite sua pergunta: Qual o status da estação 01
Resposta: 


> A estação CITY_01 apresenta um alarme crítico ativo:
> 
> * **CITY_01_XS-056A-FALHA:**  *Válvula de bloqueio - Entrada do aquecedor A - Falha * -  indica  uma falha no comando da válvula HV-056.
> 
> Este é um alarme de **prioridade crítica** e requer atenção imediata da equipe de manutenção. 
> 
> Todos os outros alarmes da estação estão em estado normal e as variáveis de pressão de entrada, pressão de saída e vazão estão dentro dos limites operacionais. 
> 
> **Em resumo:** Apesar de seus parâmetros de operação estarem dentro dos limites, a estação CITY_01 necessita de intervenção imediata para solucionar a falha crítica na válvula de bloqueio do aquecedor A. 




Digite sua pergunta: Quais ações tomar sobre o alarme crítico da estação 01
Resposta: 


> ## Ações para o Alarme Crítico CITY_01_XS-056A-FALHA:
> 
> O alarme  *CITY_01_XS-056A-FALHA * ( *Válvula de bloqueio - Entrada do aquecedor A - Falha *) indica um problema crítico que pode comprometer a segurança e a operação da estação. 
> 
> **Ações Imediatas:**
> 
> 1. **Isolar o Aquecedor A:**  Desligue o aquecedor A imediatamente para evitar qualquer dano potencial devido à falha na válvula de bloqueio.
> 2. **Verificação de Segurança:**  Garanta que a área ao redor do aquecedor A esteja segura, livre de vazamentos de gás ou outros perigos.
> 3. **Investigar a Causa da Falha:** Acionar equipe de manutenção para investigar a causa da falha no comando da válvula HV-056. Algumas possibilidades:
>     * **Falha na própria válvula:**  Verificar se a válvula está funcionando corretamente (aberta/fechada), se há algum dano físico ou obstrução.
>     * **Falha no atuador:** Verificar o atuador da válvula, incluindo fiação, alimentação e sinais de controle. 
>     * **Falha no sistema de controle:** Verificar o CLP e seus programas, buscando por erros de comunicação ou lógica de controle.
> 
> **Ações Posteriores:**
> 
> 4. **Reparar/Substituir:**  Dependendo da causa da falha, será necessário reparar ou substituir a válvula, o atuador ou componentes do sistema de controle.
> 5. **Testar a Válvula:**  Após o reparo, testar exaustivamente a válvula para garantir seu correto funcionamento.
> 6. **Reativar o Aquecedor A:**  Somente após a confirmação da resolução da falha e testes bem-sucedidos, reative o aquecedor A.
> 
> **Documentação:**
> 
> * Registrar todas as ações tomadas, os resultados das investigações e reparos realizados. 
> * Analisar a ocorrência para identificar a causa raiz da falha e implementar medidas preventivas para evitar repetições no futuro.
> 
> **Considerações:**
> 
> * A segurança é primordial. A equipe de manutenção deve seguir os procedimentos de segurança adequados durante todas as etapas.
> * A rapidez na resolução do problema é crucial para minimizar o tempo de inatividade da estação.
> * Manter contato com a equipe de operação e fornecer atualizações regulares sobre o progresso do reparo. 




Digite sua pergunta: fim
